In [1]:
import cvxpy as cp

In [2]:
x = cp.Variable()
a = cp.Parameter(nonneg=True)

In [3]:
print('Curvarture of x:', x.curvature)
print('Curvature of a:', a.curvature)
print('curvature of square(x):', cp.square(x).curvature)
print('curvature of sqrt(x):', cp.sqrt(x).curvature)

Curvarture of x: AFFINE
Curvature of a: CONSTANT
curvature of square(x): CONVEX
curvature of sqrt(x): CONCAVE


3.33

In [4]:
x = cp.Variable()
y = cp.Variable()
a = cp.Parameter()

In [5]:
sqrty   = cp.sqrt(y)
x2sqrty = cp.quad_over_lin(x, sqrty)
f = cp.norm2(1, x2sqrty)

In [6]:
f.curvature

'CONSTANT'

3.38

In [15]:
cp.minimum(2, x, cp.sqrt(x)).curvature

'CONCAVE'

**4.3** *Formulating constraints in CVX*. Below we give several convex constraints on scalar variables $x$, $y$, and $z$. Express each one as a set of valid constraints in CVX*. Directly expressing them in CVX* will lead to invalid constraints. You can also introduce additional variables, if needed.

Check your reformulations by creating a small problem that includes these constraints, and solving it using CVX*. Your test problem doesn't have to be feasible, it's enough to verify that CVX* processes your constraints without error.

**(a)** $1/x + 1/y \leq 1, x \geq 0, y \geq 0$

In [8]:
x = cp.Variable(nonneg=True)   # x >= 0
y = cp.Variable(nonneg=True)   # y >= 0

# define objective function (minimize 3*x1 + 2*x2)
objective = cp.Minimize(3*x + 2*y)

# define constraints
constraints = [
    cp.inv_pos(x) + cp.inv_pos(y) <= 1
]

# form and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

9.898979198766426

**(b)** $xy \geq 1, x \geq 0, y \geq 0$

In [9]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

objective = cp.Minimize(3*x + 2*y)

constraints = [
    x >= cp.inv_pos(y)
]

problem = cp.Problem(objective, constraints)
problem.solve()

4.89897945484331

**(c)** $(x+y)^2/\sqrt{y} \leq x - y + 5$ with implicit constraint $y \geq 0$

In [10]:
x = cp.Variable()
y = cp.Variable(nonneg=True)
s = cp.Variable(nonneg=True)
t = cp.Variable()

constraints = [
    cp.quad_over_lin(x + y, s) <= t,
    t <= x - y + 5,
    y >= cp.square(s)
]

objective = cp.Minimize(3*x + 2*y)

problem = cp.Problem(objective, constraints)
problem.solve()

-4.908788065336004

**(d)** $x + z \leq 1 + \sqrt{xy - z^2}$ $x \geq 0, y \geq 0$

In [11]:
import cvxpy as cp

x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)
z = cp.Variable()

In [17]:
import cvxpy as cp

x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)
z = cp.Variable()
t = cp.Variable(nonneg=True)

constraints = [
    x + z <= 1 + t, 
    cp.norm(cp.vstack([2 * z, 2 * t])) <= x + y, 
    t >= 0
]

objective = cp.Minimize(x + y + z)

prob = cp.Problem(objective, constraints)
prob.solve()

1.4115420131823242e-10

*Optimal activity levels*. We consider the selection of $n$ nonnegative activity levels, denoted $x_1,\dots,x_n$. These activities consume $m$ resources, which are limited. Activity j consumes
 $A_{ij}x_j$ of resource $i$, where $A_{ij}$ are given. The total resource consumption is additive, so the total of resource $i$ consumed is $c_i = \sum_{j=1}^{n}{A_{ij}x_j}$. (Ordinarily we have $A_{ij} \geq 0$, i.e., activity $j$ consumes resource $i$. But we allow the possibility that $A_{ij} < 0$, which means that activity $j$ actually generates resource $i$ as a by-product.) Each resource consumption is limited: we must have $c_i \leq c_i^{max}$, where $c_i^{max}$ are given. Each activity generates revenue, which is a piecewise-linear concave function of the activity level:
 $$
    r_j(x_j) = \begin{cases}
        p_jx_j & 0 \leq x_j \leq q_j\\
        p_jq_j + p_j^{disc}(x_j - q_j) & x_j \geq q_j.
    \end{cases}
$$

Here $p_j > 0$ is the basic price, $q_j > 0$ is the quantity discount level, and $p^{disc}$ is the quantity discount price, for (the product of) activity $j$. (We have $0 < p_j^{disc} < p_j.$) The total revenue is the sum of the revenues associated with each activity, i.e., \sum_{j=1}^n r_j(x_j). The goal is to choose activity levels that maximize the total revenue while respecting the resource limits. Show how to formulate this problem as an LP.

* Selection of $n$ nonnegative activity levels: $x_1, \dots, x_n$. 
* The activities consume $m$ resources (limited)
* Activity $j$ consumes $A_{ij}x_j$ of resource $i$.

* Total of resource $i$ consumed is $c_i = \sum_{j=1}^{n} A_{ij}x_j$ 

* resource consumption limited: $c_i \leq c_i^{max}$

Revenue generated by each activity:
$$
r_j (x_j) = \begin{cases}
    p_j x_j & 0 \leq x_j \leq q_j\\
    p_j q_j + p_j^{disc}(x_j - q_j) & x_j \geq q_j
\end{cases}
$$

* $p_j > 0$: basic price
* $q_j > 0$: quantity discount level
* $p_j^{disc}$: quantity discount price for the product of activity $j$
* $0 < p_j^{disc}$
* Total revenue associated with each activity $\sum_{j=1}^n r_j(x_j)$
* **goal:** choose activity levels that maximize the total revenue while respecting resource limits.

In [2]:
import cvxpy as cp
import numpy as np

In [ ]:
A      = cp.Parameter((5,4))    # activity x product consume
c_max  = cp.Parameter(5)        # resource maximum consume
p      = cp.Parameter(4)        # basic price
p_disc = cp.Parameter(4)        # discount price of activity j
q      = cp.Parameter(4)        # discount level 

# atribute values
A.value = np.array([[1, 2, 0, 1], 
                   [0, 0, 3, 1],
                   [0, 3, 1, 1],
                   [2, 1, 2, 5],
                   [1, 0, 3, 2]])

c_max.value  = np.full(5, 100)
p.value      = np.array([3, 2, 7, 6])
p_disc.value = np.array([2, 1, 4, 2])
q.value      = np.array([4, 10, 5, 10])

x = cp.Variable(4, nonneg=True) # activity level
r = cp.Variable(4)              # revenue vector

y = cp.Variable(4, nonneg=True)
z = cp.Variable(4, nonneg=True)

# r = p @ cp.minimum(x, q) + p_disc @ cp.maximum(x - q, 0)
r = p @ y + p_disc @ z

# objective function
objective = cp.Maximize(cp.sum(r))

# constraints
constraints = [
    A @ x <= c_max,
    y + z == x,
    y <= q,
    z >= 0
]

# form and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

Optimal activity levels (x): [ 4.00000008 22.49999987 30.99999992  1.50000002]
Optimal revenue vector (r): 192.49999976681522
Total revenue: 192.49999976681522


In [65]:
revenue = np.zeros(4)

for i in range(len(x.value)):
    if x.value[i] < q.value[i]:
        revenue[i] = p.value[i]*x.value[i]
    else:
        revenue[i] = p.value[i]*q.value[i] + p_disc.value[i]*(x.value[i] - q.value[i])

In [77]:
average_price_unit = revenue/x.value

In [78]:
print(f"Optimal activity levels (x): {[f'{v:.2f}' for v in x.value]}")
print(f"Optimal revenue vector (r): {[f'{v:.2f}' for v in revenue]}")
print(f"Total revenue: {problem.value:.2f}")
print(f"Average price per unit: {[f'{v:.2f}' for v in average_price_unit]}")

Optimal activity levels (x): ['4.00', '22.50', '31.00', '1.50']
Optimal revenue vector (r): ['12.00', '32.50', '139.00', '9.00']
Total revenue: 192.50
Average price per unit: ['3.00', '1.44', '4.48', '6.00']


*CVX implementation of a concave function*. Consider the convave fuction $f: \mathbf{R} \to \mathbf{R}$ defined by

$$
    f(x) = \begin{cases}
        (x+1)/2 & x > 1\\
        \sqrt{x} & 0 \leq x \leq 1,
    \end{cases}
$$

with $\mathbf{dom} f = \mathbf{R}_+$. Give a CVX implementation of $f$, via a partially specified optimization problem. Check your implementation by maximizing $f(x) + f(a - x)$ for several interesting values of $a$ (say, $a = -1$, $a = 1$, and $a = 3$).

In [1]:
import cvxpy as cp
import numpy as np

In [9]:
def f_cvx(x):
    f1 = (x + 1)/2
    f2 = cp.sqrt(x)

    return cp.minimum(f1, f2)

In [10]:
a_values = [-1, 1, 3]

for a in a_values:
    x = cp.Variable()
    objective = cp.Maximize(f_cvx(x) + f_cvx(a - x))
    constraints = [x >= 0, a - x >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    print(f"a = {a}, x_opt = {x.value}, f(x) + f(a - x) = {prob.value}")

a = -1, x_opt = None, f(x) + f(a - x) = -inf
a = 1, x_opt = 0.5000000000002822, f(x) + f(a - x) = 1.414213562373095
a = 3, x_opt = 1.499999999996737, f(x) + f(a - x) = 2.449489742783178
